# VQE for Fusion

In [350]:
import numpy as np
#from numpy.core.umath_tests import inner1d

import functools
from operator import mul
# from mpmath import *
import time
import sys
import random
import cmath
import itertools
import matplotlib.pyplot as plt

# Importing Qiskit libraries
from qiskit.algorithms.optimizers import ADAM, CG, GSLS, NELDER_MEAD, NFT, POWELL, SPSA, TNC, COBYLA, L_BFGS_B, SLSQP, AQGD, P_BFGS, GradientDescent
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator
from qiskit.quantum_info.operators.symplectic.pauli_utils import pauli_basis
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import SparsePauliOp
from qiskit import QuantumCircuit, transpile, Aer, IBMQ, BasicAer
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator
from qiskit.execute_function import execute
from qiskit.visualization import plot_bloch_multivector
from qiskit.quantum_info import Statevector
from qiskit.opflow import I, Z, X, Y, StateFn, SummedOp, CircuitOp, CircuitStateFn
from qiskit.opflow.gradients import Gradient, NaturalGradient, QFI, Hessian
from qiskit.circuit import QuantumCircuit, QuantumRegister, Parameter, ParameterVector, ParameterExpression
from qiskit.circuit.library import EfficientSU2

from scipy.optimize import minimize, rosen, rosen_der
import scipy
from scipy import sparse
from scipy.sparse import csr_matrix

import sympy
from sympy import Matrix
from sympy.physics.quantum import TensorProduct
from sympy.physics.quantum.dagger import Dagger

# Pre-defined ansatz circuit and operator class for Hamiltonian
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import SparsePauliOp

# The IBM Qiskit Runtime
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Estimator, Session

%config InlineBackend.figure_format='retina'

from qiskit_nature.second_q.hamiltonians.lattices import LineLattice, BoundaryCondition
from qiskit_nature.second_q.hamiltonians.ising_model import IsingModel

# Loading your IBM Quantum account(s)
# provider = IBMQ.load_account()

In [346]:
!pip install pytket pytket-qiskit pytket-quantinuum 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 7.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 224.8 kB/s eta 0:00:0000:01

[notice] A new release of pip available: 23.1.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 82.8 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.9/130.9 kB 227.9 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.9/242.9 kB 554.7 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: qiskit-ibm-provider
    Found existing installation: qiskit-ibm-provider 0.6.3
    Uninstalling qiskit-ibm-provider-0.6.3:
      Successfully uninstalled qiskit-ibm-provider-0.6.3
  Attempting uninstall: qiskit-ibm-runtime
    Found existing installation: qiskit-ibm-runtime 0.11.3
    Uninstalling qiskit-ibm-runtime-0.11.3:
      Successfully uninstalled qiskit-ibm-runtime-0.11.3

[notice] A new release of pip availab

In [356]:
from pytket.extensions.qiskit.tket_backend import TketBackend
from qiskit import Aer
from qiskit.primitives import BackendEstimator
from pytket.extensions.quantinuum import QuantinuumBackend

# # simulator = Aer.get_backend('aer_simulator')
# # aer_backend = BackendEstimator(simulator, options={"shots": 500})

# machine = 'H1-2E'
# backend_emu = QuantinuumBackend(device_name=machine)
# backend_emu.login()
# qis_backend = TketBackend(backend_emu)
# qestimator = BackendEstimator(qis_backend, options={"shots": 100})

In [358]:
!pip install qtuum

ERROR: Could not find a version that satisfies the requirement qtuum (from versions: none)
ERROR: No matching distribution found for qtuum

[notice] A new release of pip is available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [357]:
qc = QuantumCircuit(2)
qc.x(0)


from qtuum.api_wrappers import QuantinuumAPI as QAPI
import time
machine = 'H1-1E'
qapi = QAPI(machine=machine)
print('Machine status:', qapi.machine, 'is', qapi.status())

shots = 1
job_id = qapi.submit_job(openqasm, 
                         shots=shots, 
                         machine=machine, 
                         name='circuit example')


Traceback (most recent call last):
  Cell In[357], line 5
    from qtuum.api_wrappers import QuantinuumAPI as QAPI
ModuleNotFoundError: No module named 'qtuum'

Use %tb to get the full traceback.


In [375]:
II = np.array([[1,0],[0,1]])
ZZ = np.array([[1,0],[0,-1]])
XX = np.array([[0,1],[1,0]])
YY = np.array([[0,-1.j],[1.j,0]])
 
# Outputs the Kroneker product of two matrices
def kroneker_product(a,b):
    
    row = a.shape[0]*b.shape[0]    # Calculates no.of rows of the product matrix 
    col = a.shape[1]*b.shape[1]    # Calculates no.of columns of the product matrix
    
    z1 = a[:,:,None] 
    z2 = b[:,None,:]
    z3 = z1[:,None]*z2     # caries out the element-wise product in 3 and 4 dimension space(a is converted to 4 dimension matrix and b is converted to 3 Dimension matrix)  
    z3.shape = (row,col)   # reshapes the array to the dimensions calculated perviously
    return z3

# Pauli Decomposition function
def pauli_decomposition(nqubits,M): # M - Hamiltonain matrix, nqubits - no.of qubits
     
    Pauli = [II,XX,YY,ZZ]              # List of pauli gates 
    Pauli_Names = ["I","X","Y","Z"] # Names of the pauli gates 
    
    pauli_decomp = []  
    ranges = [[0,4]] * nqubits      # this is a list
    operations = 4**nqubits-1       # operations = 4^nqubits-1 e.g for 2 qubits we have 16 combinations II,IX,IY,IZ,XI,XX,.....,ZZ 
    result =[0] * nqubits 
    
    pos = nqubits-1
    inc = -1
    result[pos] = -1

    while inc < operations:
        
        if result[pos]==ranges[pos][1]-1:
            result[pos]=ranges[pos][0]
            pos-=1
    
        else:
            result[pos]+=1
            inc+=1
            pos=len(ranges)-1 
                     
            if nqubits == 1:
                P = Pauli[result[0]]
            else:
                P = kroneker_product(Pauli[result[0]],Pauli[result[1]])
                
                for ii in range(2,nqubits):
                    P = kroneker_product(P,Pauli[result[ii]])
                   
           
            coeff = (1/(2**(nqubits))) * (P*M.T).sum()                     
            if (abs(coeff) > 1e-8):                                    # this value is by default set in qiskit as 10**(-8), therefore using the same value                                      
                name = Pauli_Names[result[0]]
                
                for ii in range(1,nqubits):
                    name =  name + Pauli_Names[result[ii]]
                pauli_decomp.append([coeff,name]) 
                               
    return pauli_decomp

In [376]:
# Function definition for Hadamard test  

def Hadamard_test(U, Ansatz, qubits, Str):
    qc = QuantumCircuit(qubits+1,1)
    
    qc.append(Ansatz, list(range(1,nqubits+1)))
    
    if Str == "c":
        qc.h(0)
        qc.sdg(0)
        qc.barrier()
        qc = qc.compose(U.control(1),list(range(0,qubits+1)))
        qc.barrier()
        qc.h(0)
    
    elif Str == "r":
        qc.h(0)
        qc.barrier()
        qc = qc.compose(U.control(1),list(range(0,qubits+1)))
        qc.barrier()

        qc.h(0)
    else:
        print("Incorrect String")
        
    
    return qc

# def Psi(qubits,initial_state):
    
#     qc = QuantumCircuit(qubits, name="Psi")
#     for i in range(len(initial_state)):
#         if initial_state[i] == '1':
#             qc.x(i)
    
#     # StateVector = [1/np.sqrt(2), 1/np.sqrt(2)]
#     # qc.initialize( StateVector , list(range(qubits)))     # For Statevector
#     return qc


# Measurement methods StateVector/Matrix Multiplication/ QASM simulator 

def statevector(circ):
    sv = Statevector(circ) 
    
    probs_qubit_0 = sv.probabilities([0])
    #print('Qubit_0 probabilities for 0 and 1 are: {}\n'.format(np.round(probs_qubit_0,4)))
    p_0 = probs_qubit_0[0]
    p_1 = probs_qubit_0[1]

    return p_0-p_1

def qiskitOP(U,psi,nqubits):
    psi_=CircuitStateFn(psi)
    U_ = CircuitOp(U)
    #print(psi_.adjoint().compose(U_).compose(psi_).eval())
    return psi_.adjoint().compose(U_).compose(psi_).eval().real, psi_.adjoint().compose(U_).compose(psi_).eval().imag


def qasm(circ):             # text = "ev" - for epectation value and text = "state" for output state

    circ.measure(0,0)
   
    backend = Aer.get_backend('qasm_simulator')

    shots=1000
    result = execute(circ,backend, shots=shots).result()
    
    
    counts = result.get_counts()
    
    if len(counts) == 1:
        if "1" in counts:
            p0=0
            p1=1
        else:
            p0=1
            p1=0
    else:
        p0 = counts['0']/shots
        p1 = counts['1']/shots
    
    return p0 - p1 


In [377]:
# Calculating the expectation value of pauli string

def ExpectationOfPauli(decomp_res,nqubits,approach,Ansatz):
    ExpValuesReal = []
    
    real=0
    expVal=0
    
    for ii in range(len(decomp_res)):
        U = QuantumCircuit(nqubits)
        Str = decomp_res[ii][1]
        for i in range(len(Str)):
            if Str[i] == "X":
                U.x(i)
            elif Str[i] =="Y":
                U.y(i)
            elif Str[i] =="Z":
                U.z(i)
            else:
                pass
        
        U_gate = U.to_gate(label="U gate")
        
        circ_real = Hadamard_test(U_gate, Ansatz ,nqubits, Str="r")
        
        if approach=="statevector":   # circuit based approach. Mathematical probability output of the circuit.
            real = statevector(circ_real)
            # imaginary = statevector(circ_complex) 

        elif approach=="qiskitOP":     # Directly calculating <Psi|H|Psi> using matrix multiplication
            real ,imaginary = qiskitOP(U, psi, nqubits)

        else :        # circuit based approach. Simulating QC on classical  
            real = qasm(circ_real)
            # imaginary = qasm(circ_complex)

        ExpValuesReal.append(real)
        # ExpValuesImg.append(imaginary)
        num = real  
        
        # print(decomp_res[ii][0]*num)
        expVal += (decomp_res[ii][0]*num)
        
        
    return ExpValuesReal, expVal


def test_expectation_value(K,psi):
     
    psi_ = CircuitStateFn(psi)
    psi_ = np.matrix(psi_.eval().to_matrix())
    # print(psi_.shape)
    
    ss = Qobj(psi_)
    kk = Qobj(K)
    ss_ = Qobj(np.transpose(psi_))
    
    # print(ss.shape, kk.shape, ss_.shape)
    return ss*kk*ss_

In [378]:
# def Layer(n,theta):
    
#     count=0
#     q=QuantumCircuit(n)

#     for i in range(n):
#         for j in range(i):
#             if (i!=j):
#                 q.cx(j,i)
    
#     for i in range(n):
#         q.ry(theta[count],i)
#         count+=1
    
#     # for i in range(n):
#     #     q.rz(theta[count],i)
#     #     count+=1
    
        
#     return q

# def Ansatz(n,l):
#     num_params = int(n)
#     a = ParameterVector('a', n+(n)*l)
    
#     # print(n+(2*n-2)*l)
#     if l==0:
#         qc=QuantumCircuit(n)
#         [qc.ry(a[p],p) for p in range(n)]
#         #[qc.rz(a[n+p],p) for p in range(n)]

#     else:
#         qc=QuantumCircuit(n)
#         # Apply layer 0
#         [qc.ry(a[p],p) for p in range(n)]
#         #[qc.rz(a[n+p],p) for p in range(n)]
#         qc.barrier()
#         # Apply Layers l
#         for j in range(1,l+1):
#             par = a[n+(j-1)*num_params:(n+(j-1)*num_params)+num_params] 
            
#             layer = Layer(n,par)
            
#             qc = qc.compose(layer)
#             qc.barrier()
#     return qc, a



def Ansatz(n,l,Str ):
    if Str == 'EfficientSU2':
        qc = EfficientSU2(n, reps=l, entanglement='full', skip_unentangled_qubits=False, parameter_prefix='a')
        
    elif Str == 'Twolocal':
        qc = TwoLocal(n, ['ry','ry'], 'cx', 'full', reps=l, insert_barriers=False, parameter_prefix='a')
        
    elif Str == 'NLocal':
        qc = NLocal(n, ['ry','ry'], 'cx', 'full', reps=l, insert_barriers=False, parameter_prefix='a')    
        
    elif Str == 'RealAmplitudes':
        qc = RealAmplitudes(n, entanglement='full', reps=l, parameter_prefix='a')
        
    # elif Str == 'ExcitationPreserving':
    #     qc = ExcitationPreserving(n, reps=l, entanglement='full', skip_unentangled_qubits=False, parameter_prefix='a',mode='fsim')
        
    else:
        raise Exception("Enter correct input")
        
    return qc,qc.parameters

In [379]:
nqubits = 2
l=1
circ, num_param = Ansatz(nqubits, l, 'RealAmplitudes')

params = np.random.uniform(low=0, high=2*np.pi, size=(len(num_param),))

In [380]:
line_lattice = LineLattice(num_nodes=nqubits, boundary_condition=BoundaryCondition.OPEN)

ising_model = IsingModel(
    line_lattice.uniform_parameters(
        uniform_interaction=-1.0,
        uniform_onsite_potential=-1.0,
    ),
)

Ham_1 = ising_model.second_q_op().simplify()
H = Ham_1.to_matrix()
H.shape

(4, 4)

In [381]:
def ExpectationValue(H,Ansatz,params, nqubits):
    Ansatz = Ansatz.bind_parameters(params)
    
    decom = pauli_decomposition(nqubits,H)
    a,expVal = ExpectationOfPauli(decom,nqubits,"qasm",Ansatz)
    
    #test = test_expectation_value(H,Psi(nqubits,state_psi))
    
    return expVal


aa= ExpectationValue(H,circ,params, nqubits)

# print("Calculated Expectation value is",aa)

In [382]:
Dict = {'I':I, 'X':X, 'Y':Y, 'Z':Z}
decom = pauli_decomposition(nqubits,H)

for i in range(len(decom)):
    Str = decom[i][1]
    temp = Dict[Str[0]]
    
    for ii in range(1,len(Str)):
        temp ^= Dict[Str[ii]] 

    if i ==0:
        HH = decom[0][0] * temp
    else:
        HH += decom[i][0] * temp

def gradient(circ,a, H,nqubits,l,params,num_param, method):


        value_dict={}
        for i in range(num_param):   
            value_dict[a[i]]=params[i]

        op = ~StateFn(HH) @ CircuitStateFn(primitive=circ)

        if method=="p":
            state_grad = Gradient(grad_method='param_shift').convert(operator=op, params=a)
            state_grad_result = state_grad.assign_parameters(value_dict).eval()
        elif method=="f":
            state_grad = Gradient(grad_method='fin_diff').convert(operator=op, params=a)
            state_grad_result = state_grad.assign_parameters(value_dict).eval()
        elif method=="l":
            state_grad = Gradient(grad_method='lin_comb').convert(operator=op, params=a)
            state_grad_result = state_grad.assign_parameters(value_dict).eval()


        return [np.real(i) for i in state_grad_result]


In [383]:
def obj_fun(params):
    
    def ExpectationValue(H,Ansatz,params, nqubits):
        
        Ansatz = Ansatz.bind_parameters(params)
        decom = pauli_decomposition(nqubits,H)
        a,expVal = ExpectationOfPauli(decom,nqubits,"qasm",Ansatz)
        
        return expVal
    return ExpectationValue(H,circ,params, nqubits)

In [384]:
eigenvalues = np.linalg.eigvals(H)

In [385]:
def grad(params):
    def gradient(circ,a, H,nqubits,l,params,num_param, method):

        value_dict={}
        for i in range(num_param):   
            value_dict[a[i]]=params[i]

        op = ~StateFn(HH) @ CircuitStateFn(primitive=circ)

        if method=="p":
            state_grad = Gradient(grad_method='param_shift').convert(operator=op, params=a)
            state_grad_result = state_grad.assign_parameters(value_dict).eval()
        elif method=="f":
            state_grad = Gradient(grad_method='fin_diff').convert(operator=op, params=a)
            state_grad_result = state_grad.assign_parameters(value_dict).eval()
        elif method=="l":
            state_grad = Gradient(grad_method='lin_comb').convert(operator=op, params=a)
            state_grad_result = state_grad.assign_parameters(value_dict).eval()


        return [np.real(i) for i in state_grad_result]
        
        
    return gradient(circ,a, H,nqubits,l,params,num_param, method="p")
   
def CB(params):
    a = obj_fun(params)
    exp_val_list_real.append(a.real)

    print("Energy is:",a)
    print("------")
    
    
exp_val_list_imag = []
exp_val_list_real = []
var_list=[]
res = minimize(obj_fun, params, method="COBYLA",callback=CB, options={'tol': 1e-6,'maxiter':1000})
   
print("Quantum solution : ",np.real(res.fun))
print("Numpy solution : ",np.real(min(np.linalg.eigvals(H))))

Energy is: (-0.02250000000000002+0j)
------
Energy is: (-0.5215000000000001+0j)
------
Energy is: (-0.256+0j)
------
Energy is: (-0.24399999999999997+0j)
------
Energy is: (-0.9035000000000001+0j)
------
Energy is: (-0.9095+0j)
------
Energy is: (-0.9335+0j)
------
Energy is: (-1.0065+0j)
------
Energy is: (-1.02+0j)
------
Energy is: (-0.8699999999999999+0j)
------
Energy is: (-1.026+0j)
------
Energy is: (-0.984+0j)
------
Energy is: (-0.9994999999999999+0j)
------
Energy is: (-0.9405000000000001+0j)
------
Energy is: (-0.9924999999999999+0j)
------
Energy is: (-0.9829999999999999+0j)
------
Energy is: (-1.0135+0j)
------
Energy is: (-1.0105+0j)
------
Energy is: (-1.012+0j)
------
Energy is: (-1.03+0j)
------
Energy is: (-1.025+0j)
------
Energy is: (-1.0205+0j)
------
Energy is: (-1.0345+0j)
------
Energy is: (-1.016+0j)
------
Energy is: (-1.039+0j)
------
Energy is: (-1.0145+0j)
------
Energy is: (-1.0314999999999999+0j)
------
Energy is: (-1.043+0j)
------
Energy is: (-1.029+0j)

# Eigenvector

In [386]:
sv = np.array(Statevector(Ansatz(nqubits,l, "RealAmplitudes")[0].bind_parameters(res.x))) 
sv

array([0.55861692+0.j, 0.39732214+0.j, 0.48863467+0.j, 0.5397392 +0.j])

In [387]:
# Psi = eigenvectors[:,np.where(eigenvalues==min(eigenvalues))[0][0]]
# Psi

In [388]:
Psi = sv

In [389]:
eigenvalues

array([ 1.03077641+0.j, -0.25      +0.j, -1.03077641+0.j,  0.25      +0.j])

# Excited States

In [390]:
Project = np.outer(Psi, Psi)
for i in range(3):
    
    H = H + 5*Project
    
    res = minimize(obj_fun, params, method="COBYLA", options={'tol': 1e-6,'maxiter':1000})
    print("Quantum Excited state ",i+1," is ",res.fun)

    
    min_index = np.argmin(eigenvalues)
    eigenvalues = np.delete(eigenvalues, min_index)
    
    print("Classical Excited state ",i+1," is ",min(eigenvalues))
    print("--------------")
    Psi = np.array(Statevector(Ansatz(nqubits,l, "RealAmplitudes")[0].bind_parameters(res.x))) 
    #Psi = eigenvectors[:,np.where(eigenvalues==min(eigenvalues))[0][0]]
    
    Project += np.outer(Psi, Psi)
    


Quantum Excited state  1  is  -0.2500480421221928
Classical Excited state  1  is  (-0.2500000000000002+0j)
--------------
Quantum Excited state  2  is  0.23181132269643204
Classical Excited state  2  is  (0.24999999999999994+0j)
--------------
Quantum Excited state  3  is  0.9482871331654821
Classical Excited state  3  is  (1.0307764064044145+0j)
--------------
